<a href="https://colab.research.google.com/github/isaacmenchaca97/EfficientNetB0/blob/main/medical_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Digital Image Processing for General Image Classification: Final Project
###Isaac Menchaca Panecatl
###Dr. Ulises Moya
###Computer Science Master's
**Based on paper**: [EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks](https://arxiv.org/abs/1905.11946)  (2019), by Mingxing Tan and Quoc V. Le, introduces an innovative approach to scaling convolutional neural networks (CNNs) efficiently and effectively.

**Objective**: To develop an image classification system using the pre-trained EfficientNet-B0 model, evaluating the impact of various digital image processing techniques (blur, noise, and contrast) on the model's performance when classifying 100 general images from a selected dataset.

**Methodology**:


1.   Data Preparation: Select 100 images from a general-purpose dataset.
Perform dimensionality preprocessing to standardize the input size for the EfficientNet-B0 model.
2.   Baseline Classification: Use the pre-trained EfficientNet-B0 model to classify the original images and record metrics such as accuracy, precision, and recall.
1.   Testing with Image Processing:
  - Blur: Apply a blur filter (Gaussian Blur) to the images and perform classification using EfficientNet-B0.
  - Noise: Add random noise (salt-and-pepper) to the images and repeat the classification.
  - Contrast: Enhance image contrast using techniques such as histogram adjustment or CLAHE (Contrast Limited Adaptive Histogram Equalization), then classify the images.
2.   Results Analysis: Compare classification metrics across original and processed images.
Evaluate how each image processing technique influences the model's performance.
1.   Conclusions: Identify which image modifications positively or negatively affect classification accuracy.


**Link to video**:

##1. Data Preparation

In [1]:
#!/usr/bin/env python
## -*- coding: utf-8 -*-
__author__ = ["Isaac Menchaca Panecatl"]
__copyright__ = "Copyright 2024"
__credits__ = ["Isaac Menchaca Panecatl"]
__license__ = "MIT"
__version__ = "0.0.1"
__maintainer__ = ["Isaac Menchaca Panecatl"]
__email__ = "isaac.menchaca@edu.uag.mx"
__status__ = "Development"

import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cpu for inference


In [7]:
!pip install validators matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.3 MB/s eta 0:00:00


In [2]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -O coco_ann2017.zip

--2024-12-06 02:41:22--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.28.36, 54.231.233.217, 52.217.131.153, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.28.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘coco_ann2017.zip’

coco_ann2017.zip    100%[===================>] 241.19M  96.5MB/s    in 2.5s    

2024-12-06 02:41:24 (96.5 MB/s) - ‘coco_ann2017.zip’ saved [252907541/252907541]



In [3]:
!unzip coco_ann2017.zip

Archive:  coco_ann2017.zip
  inflating: annotations/instances_train2017.json  
  inflating: annotations/instances_val2017.json  
  inflating: annotations/captions_train2017.json  
  inflating: annotations/captions_val2017.json  
  inflating: annotations/person_keypoints_train2017.json  
  inflating: annotations/person_keypoints_val2017.json  


In [4]:
from pycocotools.coco import COCO
import requests

# Ruta al archivo de anotaciones (puedes descargarlo desde el sitio de COCO)
annFile = 'annotations/instances_val2017.json'

# Cargar el archivo de anotaciones
coco = COCO(annFile)

# Obtener los IDs de las imágenes
img_ids = coco.getImgIds()

# Generar una lista con las URLs de las imágenes
base_url = 'http://images.cocodataset.org/val2017/'  # Cambia según el conjunto de datos

uris = []

# Obtener las URLs para las primeras 100 imágenes
for img_id in img_ids[:100]:
    img_info = coco.loadImgs(img_id)[0]
    img_url = base_url + img_info['file_name']
    uris.append(img_url)

# Ver las primeras 10 URLs generadas
for uri in uris[:10]:
    print(uri)


loading annotations into memory...
Done (t=1.29s)
creating index...
index created!
http://images.cocodataset.org/val2017/000000397133.jpg
http://images.cocodataset.org/val2017/000000037777.jpg
http://images.cocodataset.org/val2017/000000252219.jpg
http://images.cocodataset.org/val2017/000000087038.jpg
http://images.cocodataset.org/val2017/000000174482.jpg
http://images.cocodataset.org/val2017/000000403385.jpg
http://images.cocodataset.org/val2017/000000006818.jpg
http://images.cocodataset.org/val2017/000000480985.jpg
http://images.cocodataset.org/val2017/000000458054.jpg
http://images.cocodataset.org/val2017/000000331352.jpg


In [8]:
efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

efficientnet.eval().to(device)

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

In [9]:
batch = torch.cat(
    [utils.prepare_input_from_uri(uri) for uri in uris]
).to(device)

##2. Baseline Classification

In [10]:
with torch.no_grad():
    output = torch.nn.functional.softmax(efficientnet(batch), dim=1)

results = utils.pick_n_best(predictions=output, n=1)

sample 0: [('shoe shop, shoe-shop, shoe store', '13.7%')]
sample 1: [('microwave, microwave oven', '86.5%')]
sample 2: [('shopping cart', '11.2%')]
sample 3: [('unicycle, monocycle', '86.1%')]
sample 4: [('mountain bike, all-terrain bike, off-roader', '63.9%')]
sample 5: [('toilet seat', '23.1%')]
sample 6: [('toilet tissue, toilet paper, bathroom tissue', '29.0%')]
sample 7: [('grille, radiator grille', '30.6%')]
sample 8: [('toilet seat', '72.3%')]
sample 9: [('toilet seat', '55.8%')]
sample 10: [('jinrikisha, ricksha, rickshaw', '20.4%')]
sample 11: [('bookshop, bookstore, bookstall', '25.7%')]
sample 12: [('prison, prison house', '27.6%')]
sample 13: [('television, television system', '43.5%')]
sample 14: [('pitcher, ewer', '38.9%')]
sample 15: [('solar dish, solar collector, solar furnace', '38.5%')]
sample 16: [('basenji', '15.5%')]
sample 17: [('park bench', '65.0%')]
sample 18: [('ox', '19.2%')]
sample 19: [('garbage truck, dustcart', '14.8%')]
sample 20: [('moving van', '58.3%